In [226]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool

In [2]:
mc_data = pd.read_json('DataSets/SciFi_200000_Clean_TrackStudy.json')

In [3]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event


In [7]:
mc_hits = mc_data.values
print('Shape MC_Hits: %s' % str(mc_hits.shape))

Shape MC_Hits: (200000, 14)


In [156]:
all_hits = np.zeros((3000000, 10))
i = 0
empty_particles = []
for hits in mc_hits:
    if (len(hits[4]) > 1):
        # Get the first element of the hit, in the first layer
        for j in range(0, len(hits[4])):
            hit = np.zeros(10)
            for k in range(0,9):
                hit[k + 1] = hits[k + 4][j]
            hit[0] = hits[0]
            all_hits[i] = hit
            i += 1
            
# Removing empty values, as some hits in the mc_hits dataset do not contain readings (PrHit...)
all_hits = all_hits[all_hits[:,0] != 0]
all_hits.shape

(2025994, 10)

In [223]:
z_last = np.unique(all_hits[all_hits[:,5] == 11][:,3])[0]
S1 = (4.5 - 0.75) / (1500 - 400)  
S2 = (0.75 - 0.75) / (1500 - 2000)

In [243]:
collected = []
hit = all_hits[0]
m = hit[2] / hit[3]
x_pred = hit[2] + m * ((z_last - hit[3]) + 120.64)
for temp in all_hits:
    if (np.abs(temp[2] - x_pred) < 280.0 and temp[1] != hit[1] and (temp[5] == 11)):
        collected.append(temp)

pool = Pool(processes=4)
#map doWork to availble Pool processes
results = pool.map(work, ( 0, 1, 2, 3))
pool.close()
pool.join()

Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-7:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7

KeyboardInterrupt: 

  File "<ipython-input-241-7a258aaba913>", line 25, in work
    if (bl < (temp_second[2] - x_pred_tmp) and (temp_second[2] - x_pred_tmp) < bh) and (temp_second[1] != temp[1]):
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt


In [241]:
def work(i):
    collected_second = []
    internal = collected[toSplit*i:toSplit*(i+1)]
    for temp in collected:
        m_tmp = (temp[2] - hit[2]) / (temp[3] - hit[3])
        x0 = hit[2] - hit[3] * m_tmp
        x_pred_tmp = hit[2] + m_tmp * (temp[3] - hit[3]) + (x0 * 2.152)
        Bl = 0
        Bh = 0
        if (x0 > 0):
            if (x0 > 400):
                bl = -S1 * (x0 - 400)
                bh = S2 * (x0 - 2000)
            if (x0 < 400):
                bl = -0.75
                bh = 0.75
        else:
            if (x0 < -400):
                bl = -S1 * (x0 + 400)
                bh = S2 * (x0 + 2000)
            if (x0 < 400):
                bl = 0.75
                bh = -0.75
        for temp_second in collected:
            if (bl < (temp_second[2] - x_pred_tmp) and (temp_second[2] - x_pred_tmp) < bh) and (temp_second[1] != temp[1]):
                collected_second.append(temp)
    return collected_second

In [188]:
for hits in mc_hits:
    if all_hits[0][1] in hits[4]:
        first_hit = hits
        break

In [216]:
first_hit[2]

[-547.6024169922,
 -546.8995361328,
 -545.967590332,
 -544.6826171875,
 -531.5939941406,
 -529.0435180664,
 -526.2611083984,
 -523.4387817383,
 -503.4122924805,
 -500.050994873,
 -496.7474060059,
 -496.7168884277,
 -493.2809753418]

In [193]:
all_collected = []
for collect in collected:
    if collect[1] in first_hit[4]:
        all_collected.append(collect)

In [197]:
np.unique(np.array(all_collected)[:,1])

array([265206., 330841., 330845., 395541., 462681., 462685.])

In [ ]:
fig = plt.figure(figsize=(19, 8))
ax = fig.add_subplot(111)

for i in range(0,1):
    X = np.array(upper_y[i][2]).astype(float)
    Y = np.array(upper_y[i][5]).astype(float)
    ax.scatter(X, Y, s=3)

ax.grid()
plt.show()

-0.05869346417475276

In [74]:
(-497.2119358809604 - mc_hits[1][5][0]) / (mc_hits[1][6][3] - mc_hits[1][6][0])

-0.1803654179532142

-558.9754804710785

In [65]:
(mc_hits[1][5][0] + m * ((8507.9990234375 - 7825.9995117188))) + (-459.3350219727 - 7825.9995117188 * m + 2.152)

-497.2119358809604

In [ ]:
for val in result:
    for hit in mc_hits:
        if (val[1] in hit[4]):
            predicted = val
            real = hit

In [ ]:
print(predicted)
print(real[4])

In [ ]:
for val in range(0, real_hits)